# 1. Exploring data

In [1]:
import os
def walk_through_dir(dir):
    for dirpaths, dirnames, filenames in os.walk(dir):
        print(f"There are {len(dirnames)} directories and {len(filenames)} images in {dirpaths}")
walk_through_dir("3k")

There are 3 directories and 3 images in 3k
There are 2 directories and 1 images in 3k/valid
There are 0 directories and 215 images in 3k/valid/images
There are 0 directories and 215 images in 3k/valid/labels
There are 2 directories and 1 images in 3k/train
There are 0 directories and 2975 images in 3k/train/images
There are 0 directories and 2975 images in 3k/train/labels
There are 2 directories and 0 images in 3k/test
There are 0 directories and 105 images in 3k/test/images
There are 0 directories and 105 images in 3k/test/labels


# 2. Training and evaluation

In [3]:
import ultralytics
import torch
from ultralytics import models

In [9]:
model = models.YOLO("yolo11s.pt")

In [10]:
torch.cuda.is_available()

True

In [ ]:
model.train(
    data='data/data.yaml',
    epochs=100,                  # ⬅️ More epochs, small datasets need more time to converge.
    imgsz=640,                   # Keep this unless your objects are tiny — then try 800.
    batch=8,                     # OK, unless VRAM allows bumping to 16 for smoother training.
    device=0,                    # Good.
    workers=2,                   # Safe.
    project='runs/train',
    name='s_3k3_100_balanced',
    lr0=0.001,                   # ⬅️ Lower initial learning rate helps avoid background bias.
    lrf=0.01,                    # Smooth decay.
    optimizer='AdamW',           # (default, but explicit.)
    close_mosaic=10,             # ⬅️ Helps model adapt to real samples toward the end.
    patience=30,                 # ⬅️ Early stopping based on no improvement.
    augment=True,                # Make sure it's on.
    cache=True                   # Speed up training.
)

In [ ]:
model = models.YOLO("runs/train/s_3k_50/weights/best.pt")
metrics = model.val(data="3k/data.yaml", split='test', batch=16,conf=0.25, iou=0.6, save_json=True)

# 3. Making predictions

In [ ]:
model = models.YOLO("runs/train/s_3k3_50/weights/best.pt")
for i in range(1,7):
    results = model.predict(source=f"samples/{i}.jpg", conf=0.05, line_width=1)
    results[0].save(filename=f'samples/{i}_pred.png')
# results[0].show()